# Model Interface Type 1: Using ACT-R's default window
    
ACT-R comes with a default window, which can be made invisible if one wants to. Using the default window is perhaps the easiest way to implement a simple task.

## The Lexical Decision task

To start, we will simulate a simple _lexical decision_ task. In the task, a string of letters appears on the screen and the model has to decide whether the string forms an english word (e.g., "zebra") or not (e.g. "xyzzy").  
    
The model is contained in the `lexical.lisp` file.

In [2]:
import actr

ACT-R connection has been started.


## Creating and initializing the window
To use ACT-R's default window, a number of initialization steps need to be done before running the model. In order, these steps are:
* Loading the model
* Creating the window, using the predefined `open_exp_window` method.
* Install the window device, using the predefined `install_device` method. This will effectively connect the window to ACT-R.
* Finally, add all the necessary elements to the window. For example, to add text, one might use `add_text_to_exp_window`

In [86]:
def setup():
    """Sets up an ACT-R experimental window. If the function is called
    while the ACT-R Environment is running, the window will be visible.
    If not, the window will be automatically made 'invisible' (visible to the 
    model but not to the experimenter)
    """
    actr.reset()
    actr.load_act_r_model("lexical.lisp")
    win = actr.open_exp_window("Test", width=400, height=400, visible=True)
    actr.install_device(win)
    actr.add_text_to_exp_window(win, "zebra", x=200, y=200, color="black")
    return win

Before running the model, we simply need to call `setup()`. To see what this window looks like, we can inspect the `win` object returned by `setup`:

In [91]:
win = setup()
win

#|Warning: No handler available for displaying a visible window.  Using a virtual window instead. |#


['vision', 'exp-window', 'Test']

So, the internal representation of the windo is just _a list of three strings_. Of course, the _real_ window object is hidden; this list is simply use as a handle for the methods in the `actr.py` file to keep track of which visual objects exists and how they should be visible to ACT-R.  
## The visicon
At this point, we need to talk about ACT-R way of seeing things. ACT-R follows an old-fashioned model of visual attention developed by Anne Treisman. In this model, elementray visual features are immediately available to the visual module without moving attention. These features correspond to specific attributes (e.g., color, orientation, shape), and their position on the visual field. The distribution of these elements in the visual field is called the __visicon__. The visicon can be inspected an any time. 

In [92]:
actr.run_n_events(2)  # This is needed to let ACT-R's scheduler process the window first.
actr.print_visicon()

     0.000   ------                 Stopped because event limit reached
Name              Att  Loc             TEXT  KIND  COLOR  WIDTH  VALUE    HEIGHT  SIZE        
----------------  ---  --------------  ----  ----  -----  -----  -------  ------  ----------
VISUAL-LOCATION0  NEW  (519 506 1080)  T     TEXT  BLACK  35     "zebra"  10      0.97999996


So, the visicon is just a list of features. These features are represented, by default, in a special chunk type called _visual-location_, each of which contains some helpful attributes such as whether the object contains text (T/NIL), its color, its location, and its size. 

With the visicon now in place, the model can process the screen.

In [93]:
actr.run(0.3)

     0.000   VISION                 SET-BUFFER-CHUNK VISUAL-LOCATION VISUAL-LOCATION0 NIL
     0.000   VISION                 visicon-update
     0.000   PROCEDURAL             CONFLICT-RESOLUTION
     0.050   PROCEDURAL             PRODUCTION-FIRED LOOK-AT-STRING
     0.050   PROCEDURAL             CLEAR-BUFFER VISUAL-LOCATION
     0.050   VISION                 Find-location
     0.050   VISION                 SET-BUFFER-CHUNK VISUAL-LOCATION VISUAL-LOCATION0
     0.050   PROCEDURAL             CONFLICT-RESOLUTION
     0.100   VISION                 Move-attention VISUAL-LOCATION0-1
     0.100   PROCEDURAL             CONFLICT-RESOLUTION
     0.185   VISION                 Encoding-complete VISUAL-LOCATION0-1 NIL
     0.185   VISION                 SET-BUFFER-CHUNK VISUAL TEXT0
     0.185   PROCEDURAL             CONFLICT-RESOLUTION
     0.235   PROCEDURAL             PRODUCTION-FIRED DECIDE
     0.235   PROCEDURAL             CLEAR-BUFFER RETRIEVAL
     0.235   DECLARATIVE          

[0.3, 42, None]

## Finding and Attending Objects 

If you look at the `lexical.lisp` file, its entire visual processing is made of two productions, `look-at-string` and `decide`. The first one, `look-at-string`, finds a specific text object in the visicon by asking the `visual-location` buffer to find any object that has the property `kind text`m (i.e., it is a text). This will put the corresponding visual-location object in the `visual-location` buffer. Because we have set the `auto-attend` feature to `t`, everything that ends up in the `visual-location` buffer is also processed in the `visual` buffer.

So, let's inspect the contents of this buffer:

In [95]:
# Inspects the visual buffer
actr.buffer_chunk("visual")

VISUAL: TEXT0-0 [TEXT0]
TEXT0-0
   SCREEN-POS  VISUAL-LOCATION0-1
   VALUE  "zebra"
   COLOR  BLACK
   HEIGHT  10
   WIDTH  35
   TEXT  T


['TEXT0-0']

# Model Interface Type 2: Defining your own visual features

The experimental window is pretty limited. It can add only text and, by default, the text is transformed into special objects that only record the color and the text string (as a string, not even as chunk). It can add horizontal and vertical lines. With some effort, it can add images, but not truly an image description.

What if you needed to encode much more infromation for a single word, such as font, and size? What if you wanted your visual objects to be immediately recognized as words or nonwords? That, unfortunately, cannot be done.

To create complex interfaces, a programmer must be able to directly act on the visicon, generating visual locations and visual objects as they please.

To do so, we will will consider a similar task. Instead of judging whether a string is a word or a nonw-word, the new model will look at a visual stimulus and decided whether the object represented in the picture is natural or man-made.